In [1]:
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Refat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Refat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
#Reading the data
import re
import tkinter as tk
from collections import defaultdict

with open('./corpus.txt', 'r') as file:
    corpus = file.read()
       
corpus = re.sub(r'[.,\'"():;’-]', '', corpus) #removing special characters
corpus = corpus.lower()   
tokens = nltk.word_tokenize(corpus)

In [28]:
# divide the tokens into a bunch of n-grams
def ngram(tokens, n):
    finalResult = []
    for i in range(len(tokens) - n + 1):
        myList = tokens[i:i+n]
        myTuple = tuple(myList)
        finalResult.append(myTuple)
    return finalResult


# Count the frequency of each n-gram
def countFreq(nGrams, n):
    nGramFreq = defaultdict(lambda: defaultdict(lambda: 0)) #initialization for a 2d dictionary
    for words in nGrams:
        start = words[0:n-1]
        end = words[-1]
        nGramFreq[start][end] += 1
    return nGramFreq


# Calculate the probability of each n-gram
def calcProbability(nGramFreq, n):
    for start in nGramFreq:
        totalNGrams = float(sum(nGramFreq[start].values()))
        for end in nGramFreq[start]:
            nGramFreq[start][end] /= totalNGrams
    return nGramFreq

In [29]:
def createTheModel(token, n):
    nGrams = ngram(tokens, n)
    nGramFreq = countFreq(nGrams, n)
    model = calcProbability(nGramFreq, n)
    return model

In [32]:
#running n-gram
n = input("Enter n: ")
sentence = input("write your words: ") 

while True:
    sentenceTokens = nltk.word_tokenize(sentence.lower())
    
    if(len(sentenceTokens) == 1):
        n = 2
    else:
        n = 3

    model = createTheModel(tokens, n)
    sentenceTokens = tuple(sentenceTokens[-n+1:])
    for start in model:
        if sentenceTokens == start:
            i=1
            predictions = sorted(model[start].items(), key = lambda t:(-t[1])) #-t[1] is to sort in a descending order
            for predictedWord in predictions:
                if i == 11:
                    break
                print(i,"-",predictedWord[0]) 
                i += 1
    answer = input("Want to terminate? y or n : ")
    if answer == 'y' or answer == 'Y':
        break;
    sentence += " " + input(">>> " + sentence + " ")

Enter n: 5
write your words: you
1 - can
2 - have
3 - are
4 - know
5 - think
6 - dont
7 - need
8 - that
9 - and
10 - will
Want to terminate? y or n : y


In [34]:
#GUI
def suggestNextWord(userInput):
    global n
    userInput = userInput.lower()
    userInputTokens = nltk.word_tokenize(userInput)
    if len(userInputTokens) == 1:
        n = 2
    else:
        n = 3
    model = createTheModel(tokens, n)
    userInputTokens = tuple(userInputTokens[-n+1:]) #take the last n-1 words
    for start in model:
        if userInputTokens == start:
            i = 1
            predictions = sorted(model[start].items(), key=lambda t:(-t[1])) #-t[1] is to sort in a descending order
            for predictedWord in predictions:
                if i == 10:
                    break
                yield predictedWord[0] #[0] to get the key only not the probability, return it as an object
                i += 1


def onkeyRelease(event):
    global searchBar, output
    inputText = searchBar.get()
    output.config(state = tk.NORMAL) 
    output.delete('1.0', "end")
    if inputText:
        if(inputText[-1] == ' '):
            inputText = inputText[:-1]
        for suggestion in suggestNextWord(inputText):
            output.insert("end", inputText + " " + suggestion + "\n")
    output.config(state = tk.DISABLED) 


# Create the main window
window = tk.Tk()
window.title("You type, We Complete!")

# Create the search bar
searchBar = tk.Entry(window, width = 54)
searchBar.pack(pady = 10)    
searchBar.bind('<KeyRelease>', onkeyRelease)

# Create the output widget
output = tk.Text(window, height = 10, width = 70)
output.pack()

window.mainloop()